# JSON Schema Form
Powered by [rjsf](https://github.com/rjsf-team/react-jsonschema-form).

In [ ]:
from yaml import safe_dump
import ipywidgets as W, traitlets as T
from wxyz.json_schema_form import JSONSchemaForm
from wxyz.yaml import YAML
from wxyz.lab import DockBox, Editor
from IPython.display import Markdown

In [ ]:
some_schema = """
title: Quest
description: Answer me these riddles 3... or 4... or...
type: array
items:
    $ref: "#/definitions/party-member"

definitions:
    party-member:
        type: object
        required:
            - your_name
            - your_quest
            - your_favorite_color
        properties:
            your_name:
                title: Name
                description: What is your name?
                type: string
                pattern: ^(S[ei]r|Dame|Lady|King|Queen|Prince)
            your_quest:
                title: Quest
                description: What is your Quest?
                type: string
            your_favorite_color:
                title: Color
                description: What is your favorite color?
                type: string
                enum:
                    - red
                    - green
                    - blue
                    - AAAAH
            asv:
                title: Velocity
                description: What is the airspeed velocity of an unladen swallow?
                anyOf:
                    - type: number
                      title: Oh, that's easy...
                    - type: string
                      title: What kind of swallow?
                      enum:
                          - african
                          - european
""".strip()

In [ ]:
some_ui_schema = """
items:
  your_quest:
    ui:widget: textarea
    ui:options:
      rows: 3
  your_favorite_color:
    ui:widget: radio
    
""".strip()

In [ ]:
some_layout = {'type': 'split-area',
 'orientation': 'horizontal',
 'children': [{'type': 'split-area',
   'orientation': 'vertical',
   'children': [{'type': 'tab-area', 'widgets': [0], 'currentIndex': 0},
    {'type': 'tab-area', 'widgets': [1], 'currentIndex': 0}],
   'sizes': [1, 1]},
  {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0},
  {'type': 'split-area',
   'orientation': 'vertical',
   'children': [{'type': 'tab-area', 'widgets': [3], 'currentIndex': 0},
    {'type': 'tab-area', 'widgets': [4], 'currentIndex': 0}],
   'sizes': [1, 1]}],
 'sizes': [1, 1, 1]}

In [ ]:
def make_a_json_schema_form_playground(schema=some_schema, ui_schema=some_ui_schema):
    schema_editor = Editor(description="Schema", value=some_schema, config=dict(mode="yaml", theme="zenburn"))
    ui_schema_editor = Editor(description="UI Schema", value=some_ui_schema, config=dict(mode="yaml", theme="zenburn"))
    form = JSONSchemaForm(description="Form")
    schema_yaml = YAML()
    ui_schema_yaml = YAML()
    instance_yaml = W.Output()
    error_yaml = W.Output()
    
    W.jslink((schema_editor, "value"), (schema_yaml, "source"))
    W.jslink((schema_yaml, "value"), (form, "schema"))

    W.jslink((ui_schema_editor, "value"), (ui_schema_yaml, "source"))
    W.jslink((ui_schema_yaml, "value"), (form, "ui_schema"))
    
    def _on_form_data(change):
        instance_yaml.clear_output()
        with instance_yaml:
            display(Markdown(f"```yaml\n{safe_dump(form.value)}```"))
    form.observe(_on_form_data, "value")

    def _on_errors(change):
        error_yaml.clear_output()
        with error_yaml:
            display(Markdown(f"```yaml\n{safe_dump(form.errors)}```"))
    form.observe(_on_errors, "errors")
    
    box = DockBox(
        [schema_editor, ui_schema_editor, form, instance_yaml, error_yaml], 
        layout=dict(height="80vh"), 
        dock_layout=some_layout
    )
    instance_yaml.add_traits(description=T.Unicode("Instance").tag(sync=True))
    error_yaml.add_traits(description=T.Unicode("Errors").tag(sync=True))
    
    return form, box

In [ ]:
if __name__ == "__main__":
    form, box = make_a_json_schema_form_playground()
    display(box)